In [14]:
import json
import glob
import cv2
import os
import gzip
import numpy as np
from scipy import interpolate
import subprocess

In [2]:
class TobiiSegment:
    def __init__(self):
        self.namedir = ''
        self.segnum = ''
        self.gd = {}
        self.pc = {}
        self.pd = {}
        self.gidx = {}
        self.gp = {}
        self.gp3 = {}
        self.vts2ts = None
        self.pts2ts = None
        
    def load(self,rec,ns):
        vts = []
        vts_ts = []
        pts = []
        pts_ts = []        
        
        # load data from file from recording directory and segment number
        with gzip.open(os.path.join(*[rec,'segments','%d'%(ns),'livedata.json.gz']), mode='rt', encoding='utf-8') as f_json:
            for line in f_json:
                # load single line data
                d = json.loads(line)
                
                if 'ts' not in d.keys():
                    continue
                    
                ts = d['ts']
                
                # for vts,pts -> ts conversion function

                
                # vts and pts
                if 'vts' in d.keys():
                    vts.append(d['vts'])
                    vts_ts.append(ts)
                elif 'pts' in d.keys():
                    pts.append(d['pts'])
                    pts_ts.append(ts)                    
                # data
                elif 'gd' in d.keys():
                    self.gd[ts] = {d['eye']: d['gd'], 's': d['s']}
                elif 'pc' in d.keys():
                    self.pc[ts] = {d['eye']: d['pc'], 's': d['s']}
                elif 'pd' in d.keys():
                    if ts not in self.pd.keys():
                        self.pd[ts] = {d['eye']: d['pd'], 's': d['s']}
                    else:
                        self.pd[ts].update({d['eye']: d['pd'], 's': d['s']})
                elif 'gd' in d.keys():
                    self.gd[ts] = {d['eye']: d['gd'], 's': d['s']}
                elif 'gp' in d.keys():
                    self.gp[ts] = {'data': d['gp'], 's': d['s']}
                elif 'gp3' in d.keys():
                    self.gp3[ts] = {'data': d['gp3'], 's': d['s']}
                    
            # Get Interpolation Function
            self.vts2ts = interpolate.interp1d(np.array(vts), np.array(vts_ts), fill_value='extrapolate')
            self.pts2ts = interpolate.interp1d(np.array(pts), np.array(pts_ts), fill_value='extrapolate')
                    
    def find_gaze_ts(self,vts):
        ti = self.vts2ts(vts)
        idx = np.abs(np.asarray(list(self.gp.keys())) - ti).argmin()
        #print(ti,'-->',list(self.gp.keys())[idx],ti-list(self.gp.keys())[idx])
        return list(self.gp.keys())[idx]
        

In [32]:
if __name__ == '__main__':
    
    # Load option file
    with open('option.json','r',encoding='utf-8') as f_json:
        opt = json.load(f_json)
    
    DIR_PROJ = opt['DIR_PROJ']
    
    # Load Project description file
    f_json = open(os.path.join(DIR_PROJ,'project.json'),'r')
    pj = json.load(f_json)['pr_info']

    print('ProjectName: ' + pj['Name'])
    print('Created    : ' + pj['CreationDate'])

    # read all recordings
    RECS = glob.glob(os.path.join(*[DIR_PROJ,'recordings','*']))

    for rec in RECS:
        f_json  = open(os.path.join(*[rec,'recording.json']),'r')
        recordings = json.load(f_json)

        if recordings is None:
            continue
            
        f_json  = open(os.path.join(*[rec,'participant.json']),'r')
        participant = json.load(f_json)

        print('----')
        print(' Dir :' + rec)
        print(' Name:' + recordings['rec_info']['Name'])
        print(' Date:' + recordings['rec_created'])
        print(' Participant:' + participant['pa_info']['Name'])

        # for making gaze video (without audio)
        for ns in range(recordings['rec_segments']):
            # output file check
            infile  = os.path.join(*[rec,'segments','%d'%(ns+1),'fullstream.mp4'])
            outfile = os.path.join(*[DIR_PROJ,'%s-%s-%d.mp4'%(recordings['rec_info']['Name'],participant['pa_info']['Name'],ns+1)])
            if os.path.exists(outfile):
                print('File', outfile, 'exists.\nbreak.')
                continue               
            
            # read livedata
            livedata = TobiiSegment();
            livedata.load(rec, ns+1)

            # read mp4 from video
            cap = cv2.VideoCapture(infile)
            fps = cap.get(cv2.CAP_PROP_FPS)
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            
            # video output settings
            fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
            vout = cv2.VideoWriter(outfile, fourcc, fps, (width,height))       
            
            # start processing
            n = 0
            gx = []
            gy = []
            gp_draw_duration = 1
            
            while(True):
                ret, frame = cap.read()

                if frame is None:
                    break

                if n > 0:
                    ti = int(n*1000000.0/fps)
                else:
                    ti = 0

                ts = livedata.find_gaze_ts(ti)
                gp = livedata.gp[ts]
                
                # set gx(n),gy(n) for dummy data
                gx.append(-1)
                gy.append(-1)
                
                try:                    
                    pupil_L = livedata.pd[ts]['left']
                    pupil_R = livedata.pd[ts]['right']
                except:
                    pupil_L = 0
                    pupil_R = 0
            
                # Draw gaze point
                if gp['s'] == 0:
                    gx[n] = int(gp['data'][0] * width)
                    gy[n] = int(gp['data'][1] * height)

                    if n > gp_draw_duration:
                        for t in range(gp_draw_duration):
                            if gx[n-t] > 0:
                                val = int(255*(gp_draw_duration-t)/gp_draw_duration)
                                rad = int(25*(gp_draw_duration-t)/gp_draw_duration)
                                cv2.circle(frame, (gx[n-t],gy[n-t]), rad, (0,val,val), thickness=7, lineType=cv2.LINE_AA)
                    else:
                        cv2.circle(frame, (gx[n],gy[n]), 25, (0,255,255), thickness=5, lineType=cv2.LINE_AA)
                        
                    cv2.putText(frame, 'Gaze: %d, %d   Pupil diameter: %f, %f'%(gx[n],gy[n],pupil_L,pupil_R),
                                (12,55), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 2)                        
                    
                # Write file info
                cv2.putText(frame, 'Name: %s  Date: %s  Participant: %s'%(recordings['rec_info']['Name'],recordings['rec_created'],participant['pa_info']['Name']), (12,25), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 2)
                
                cv2.imshow('monitor', frame)
                
                vout.write(frame)
                
                k = cv2.waitKey(10)

                if k == 27:
                    # ESC to quit
                    cv2.destroyAllWindows()
                    break
                    
                n += 1

            cap.release()
            vout.release()
            cv2.destroyAllWindows()
            
        # copy audio stream
        for ns in range(recordings['rec_segments']):
            # output file check
            infile1  = os.path.join(*[rec,'segments','%d'%(ns+1),'fullstream.mp4'])
            infile2 = os.path.join(*[DIR_PROJ,'%s-%s-%d.mp4'%(recordings['rec_info']['Name'],participant['pa_info']['Name'],ns+1)])            
            outfile = os.path.join(*[DIR_PROJ,'%s-%s-%d-a.mp4'%(recordings['rec_info']['Name'],participant['pa_info']['Name'],ns+1)])

            if os.path.exists(outfile):
                print('File', outfile, 'exists.\nbreak.')
                continue
            
            # extract audio file
            cmd = opt['FFMPEG'] + ' -i ' + infile1 + ' ' + opt['FFMPEG_OPT_AUDIO'] + ' ' + os.path.join(*[rec,'tmp.mp3'])
            print('Running.. ',cmd)
            os.system(cmd)
            
            # merge to video file
            cmd = opt['FFMPEG'] + ' -i ' + infile2 + ' -i ' + os.path.join(*[rec,'tmp.mp3']) + ' ' + opt['FFMPEG_OPT_MERGE'] + ' ' + outfile
            print('Running.. ',cmd)
            subprocess.run(cmd)            

ProjectName: humanitude_ishigaki
Created    : 06/21/2020 22:25:16
----
 Dir :D:\2020.石垣島データ\tobii\projects\vp5ubda\recordings\2cymzdy
 Name:Recording012
 Date:2020-06-24T07:08:00+0000
 Participant:ginest
File D:\2020.石垣島データ\tobii\projects\vp5ubda\Recording012-ginest-1.mp4 exists.
break.
Running..  D:\2020.石垣島データ\ffmpeg-win64\bin\ffmpeg.exe -i D:\2020.石垣島データ\tobii\projects\vp5ubda\recordings\2cymzdy\segments\1\fullstream.mp4 -vn -ac 2 -f mp3 -af aresample=async=1 D:\2020.石垣島データ\tobii\projects\vp5ubda\recordings\2cymzdy\tmp.mp3
Running..  D:\2020.石垣島データ\ffmpeg-win64\bin\ffmpeg.exe -i D:\2020.石垣島データ\tobii\projects\vp5ubda\Recording012-ginest-1.mp4 -i D:\2020.石垣島データ\tobii\projects\vp5ubda\recordings\2cymzdy\tmp.mp3 -c:v copy -c:a copy -map 0:v:0 -map 1:a:0 D:\2020.石垣島データ\tobii\projects\vp5ubda\Recording012-ginest-1-a.mp4
----
 Dir :D:\2020.石垣島データ\tobii\projects\vp5ubda\recordings\3xfq5dr
 Name:Recording003
 Date:2020-06-22T04:37:52+0000
 Participant:genest
File D:\2020.石垣島データ\tobii\project

----
 Dir :D:\2020.石垣島データ\tobii\projects\vp5ubda\recordings\pj35ead
 Name:Recording016
 Date:2020-06-24T07:35:15+0000
 Participant:Dr.Imamura
File D:\2020.石垣島データ\tobii\projects\vp5ubda\Recording016-Dr.Imamura-1.mp4 exists.
break.
Running..  D:\2020.石垣島データ\ffmpeg-win64\bin\ffmpeg.exe -i D:\2020.石垣島データ\tobii\projects\vp5ubda\recordings\pj35ead\segments\1\fullstream.mp4 -vn -ac 2 -f mp3 -af aresample=async=1 D:\2020.石垣島データ\tobii\projects\vp5ubda\recordings\pj35ead\tmp.mp3
Running..  D:\2020.石垣島データ\ffmpeg-win64\bin\ffmpeg.exe -i D:\2020.石垣島データ\tobii\projects\vp5ubda\Recording016-Dr.Imamura-1.mp4 -i D:\2020.石垣島データ\tobii\projects\vp5ubda\recordings\pj35ead\tmp.mp3 -c:v copy -c:a copy -map 0:v:0 -map 1:a:0 D:\2020.石垣島データ\tobii\projects\vp5ubda\Recording016-Dr.Imamura-1-a.mp4
----
 Dir :D:\2020.石垣島データ\tobii\projects\vp5ubda\recordings\s3l4gsi
 Name:Recording019
 Date:2020-06-24T11:00:31+0000
 Participant:ginest
File D:\2020.石垣島データ\tobii\projects\vp5ubda\Recording019-ginest-1.mp4 exists.
break.